# Factor Analysis

## Problem 2: Convex low rank matrix factorization
Solve using proximal gradient descent the following problem,
$$\min_X \frac{1}{2}\|R - X\|_F^2 + \lambda\|X\|_*$$

We will use the following facts to set up the proximal gradient descent algorithm.

1) $\|X\|_{*} = \sum_{i = 1}^n \sigma_i$; the nuclear norm of X is the sum of singular values of X.

2) For $g(X) = ||X||_{*}$, 
$\mathrm{prox}_{\kappa g}(Z) = U \hat \Sigma V^T$, where $\hat \Sigma$ is a $n \times n$ diagonal matrix with $i$th diagonal entry equal to $\hat \sigma_i$ where 
$$
\hat \sigma_i = 
\begin{cases}
 \sigma_i - \kappa, \mbox{ when }\sigma_i > \kappa \\
0  \mbox{ otherwise }
\end{cases}
$$
Here $U$ and $V$ are such that the SVD of $Z = U \Sigma V^T$

In [1]:
import random
import numpy as np 
from numpy import linalg as LA

np.random.seed(123)
m = 500
n = 200
R = np.random.randn(m, n)
regularizing_constant = 30

In [2]:
# Define functions

# Compute the objective function
def cvx_mf_obj(X):
    U, s, V = np.linalg.svd(X, full_matrices=False)
    #print s
    return 0.5*LA.norm(R - X, 'fro')**2 + regularizing_constant*np.sum(s)

# Compute the gradient of smooth part of objective functon
def cvx_mf_smooth_grad(X):
    return (X-R)

# Compute the prox of nuclear norm
def prox_nuclear_norm(Z, kappa):
    U, s, V = np.linalg.svd(Z, full_matrices=False)
    prox_vector = np.zeros(len(s))
    for i in range(len(s)):
        if s[i] > kappa:
            prox_vector[i] = s[i] - kappa
    return np.dot(np.dot(U, np.diag(prox_vector)), V.T)

In [3]:
# Proximal gradient method (Q.2, part 3)

tol = 1e-6
itm = 10000
err = float('Inf')
X = R
X_old = X
step_size =  1.0 # step size is  1/Beta and Beta is 1 in our case.

for i in range(itm):
    X_old = X
    
    # Update X
    g = cvx_mf_smooth_grad(X)
    X = prox_nuclear_norm(X - step_size*g, regularizing_constant*step_size)
    
    #Update convergence history
    obj = cvx_mf_obj(X)
    err = LA.norm(X - X_old, 'fro')/ step_size
    print "iteration number %.1f, obj is %f, error is %f" % (i, obj, err)
    if err < tol:
        break

iteration number 0.0, obj is 52415.732739, error is 316.617817
iteration number 1.0, obj is 52415.732739, error is 0.000000


In [4]:
# Analysis of the result (Q.2, part 4)

k = LA.matrix_rank(X) 
U, s, V  = np.linalg.svd(R, full_matrices=False)
U = U[:,:k]
V = V[:k,:]
s = s[:k]
X_k= np.dot(np.dot(U,np.diag(s)), V)

In [5]:
k

29

## Problem 3: Nonconvex low rank matrix factorization
Consider the low rank variation of the low rank matrix factorization,
$$
\min_{B, F} \frac{1}{2}\|R - B F\|_F^2,\quad \text{s.t. }B \in R^{m\times k},\, F\in R^{k \times n}.
$$
where $k$ is the rank we obtain from Q.2 Part 3). Use PALM algorithm to solve this optimization problem.

In [6]:
# Compute the objective function

def noncvx_mf_obj(B,F):
     return 0.5*LA.norm(R - np.dot(B, F), 'fro')**2

# Compute the gradient w.r.t B of the objective function
def noncvx_mf_grad_B(B,F):
    return np.dot(np.dot(B, F) - R, F.T)

# Compute the gradient w.r.t F of the objective function
def noncvx_mf_grad_F(B,F):
    return np.dot(B.T, np.dot(B, F) - R)   

In [7]:
# PALM algorithm

tol = 1e-6
itm = 20000
err = float('Inf')
np.random.seed(123)

B = np.random.randn(m,k)
B_old = B
F = np.random.randn(k,n)
F_old = F

for i in range(itm +1):
    
    # update B  
    eta_B = 0.0 
    beta_B = LA.norm(np.dot(F_old, F_old.T))
    eta_B = 1.0/ beta_B
    g_B = noncvx_mf_grad_B(B_old ,F_old)
    B = B_old - eta_B*g_B
    
    # update F
    eta_F = 0.0
    beta_F = LA.norm(np.dot(B.T, B))
    eta_F = 1.0/ beta_F
    g_F = noncvx_mf_grad_F(B, F_old)
    F = F_old - eta_F*g_F

    # Update convergence history
    err = LA.norm(B - B_old, 'fro') + LA.norm(F - F_old, 'fro')
    obj = noncvx_mf_obj(B, F)
    B_old = B
    F_old = F
    
    if i%1000 == 0:
        print "iteration number %.1f, obj is %f, error is %f" % (i, obj, err)
    if err < tol:   
        break

iteration number 0.0, obj is 675337.856440, error is 35.616647
iteration number 1000.0, obj is 34728.428063, error is 0.039273
iteration number 2000.0, obj is 34547.816949, error is 0.013946
iteration number 3000.0, obj is 34515.366496, error is 0.007723
iteration number 4000.0, obj is 34501.878548, error is 0.005883
iteration number 5000.0, obj is 34493.487429, error is 0.004671
iteration number 6000.0, obj is 34488.427610, error is 0.003535
iteration number 7000.0, obj is 34485.558344, error is 0.002665
iteration number 8000.0, obj is 34483.836572, error is 0.002133
iteration number 9000.0, obj is 34482.614441, error is 0.001881
iteration number 10000.0, obj is 34481.557282, error is 0.001814
iteration number 11000.0, obj is 34480.507371, error is 0.001839
iteration number 12000.0, obj is 34479.406929, error is 0.001886
iteration number 13000.0, obj is 34478.262782, error is 0.001908
iteration number 14000.0, obj is 34477.123326, error is 0.001880
iteration number 15000.0, obj is 344

In [9]:
# Compare the result of non convex low rank approximation and convex low rank approximation

LA.norm(X_k - np.dot(B, F), 'fro')/ LA.norm(X_k, 'fro')
#LA.norm(X_k - np.dot(B, F), 'fro')

0.067407916261938011

There is only about 6% change.

## Problem 4: Robust Risk Decomposition
Consider the problem
$$\min_{B,F} \rho_\kappa(\bar{R} - BF), \quad\text{s.t. }B^TB = I.$$

In [10]:
# load data and set parameters

N   = 499
T   = 251
fid = open("returns.bin", "rb")
R = np.fromfile(fid, np.float64)
fid.close()
R = np.reshape(R, (T, N)).T

# Standardize each row of R by subtracting the mean
Y = R- R.mean(axis=1, keepdims=True)
kappa = 1.0
itm = 8000
tol = 1e-6

In [11]:
# Define functions

def huber(r):
    val = 0.0;
    for row in r:
        for elem in row:
            a = np.absolute(elem)
            if a > kappa:
                val += kappa*a - 0.5*kappa**2
            else:
                val += 0.5*a**2
    return val

def f_robust(B,F):
    return huber(Y - np.dot(B, F))

# Compute the Huber derivative and vectorize it 
# to apply over each element of a matrix.

def huber_derivative(x):
    if x < -kappa:
        return -kappa
    elif x > kappa:
        return kappa
    else:
        return x
    
huber_derivative = np.vectorize(huber_derivative)

def grad_B(B,F):
    r = Y - np.dot(B, F)
    return -np.dot(huber_derivative(r), F.T)
 
def grad_F(B,F):
    r = Y - np.dot(B, F)
    return -np.dot(B.T, huber_derivative(r))

def prox_nearestorthonormal(Z):
    U, s, V = np.linalg.svd(Z, full_matrices=False)
    return np.dot(U, V.T)    

In [12]:
# PALM algorithm

itm = 8000
tol = 1e-6
err = float('Inf')

np.random.seed(123)
B = np.eye(N, k)
B_old = B

F = np.random.randn(k,T)
F_old  = F

for i in range(itm):
    
    # Update B
    eta_B = 0.0 
    beta_B = LA.norm(np.dot(F_old, F_old.T))
    eta_B = 1.0/ beta_B
    gr_B = grad_B(B_old,  F_old)
    B = prox_nearestorthonormal(B_old - eta_B*gr_B)
    
    # Update F. Note that eta_F is always 1 since beta_F = ||B^TB||_2 = 1
    eta_F = 1.0
    gr_F = grad_F(B, F_old)
    F = F_old - eta_F*gr_F
    
    # Update convergence  history 
    err = LA.norm(B - B_old, 'fro') + LA.norm(F - F_old, 'fro')
    
    obj = f_robust(B, F)
    B_old = B
    F_old = F
    
    if i%500 == 0:
        print "iteration  number %.1f, obj is %f, error is %f" % (i, obj, err)
    if err < tol:
        break

iteration  number 0.0, obj is 54921.841992, error is 84.045383
iteration  number 500.0, obj is 39107.775804, error is 151.520358
iteration  number 1000.0, obj is 38848.455328, error is 147.442987
iteration  number 1500.0, obj is 39727.879453, error is 150.308423
iteration  number 2000.0, obj is 39729.766647, error is 151.049622
iteration  number 2500.0, obj is 40060.451172, error is 151.717607
iteration  number 3000.0, obj is 40201.702201, error is 151.897731
iteration  number 3500.0, obj is 40047.377317, error is 153.711456
iteration  number 4000.0, obj is 40259.764243, error is 149.596692
iteration  number 4500.0, obj is 39681.626930, error is 149.364574
iteration  number 5000.0, obj is 40706.403203, error is 155.081662
iteration  number 5500.0, obj is 39309.258399, error is 150.096454
iteration  number 6000.0, obj is 39487.439653, error is 149.488040
iteration  number 6500.0, obj is 39938.489721, error is 148.019159
iteration  number 7000.0, obj is 39851.280803, error is 149.054242


In [13]:
b1 = B[:,0]
ind = np.argmax(np.absolute(b1))
ind

29